<a href="https://colab.research.google.com/github/GiorgiatolfoBL/docx2tabs/blob/main/Docx2dataframe_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This tutorial will teach you how to transform a docx into a pandas dataframe and then save it as a CSV file.

Prerequisites:
* you will need Python installed on your computer
* make sure that your file is a .docx and not a .doc

## Install the appropriate Python packages

In [ ]:
#specific to extracting information from word documents
import os
import zipfile

#other tools useful in extracting the information from our document
import re

#to read XML and JSON
from lxml import etree
import json

#to use dataframes
import pandas as pd

## Import the file
* On the left side of colab, hover over the "folder" icon.
* Click on the import icon and select your file.
* Make sure your file is saved in the sample_data folder (if not, just drag and drop it inside it)






## Anatomy of a .docx file

What is a docx file? Well, it is in fact an archive of xml files!
Let's look at it.

In your local machine:
- Rename the file extension from .docx to .zip 
- Unzip the newly renamed file

Ok now we are ready.
In the next cell look at 

```
file = "sample_data/india_sample"
```

Make sure that the file you have uploaded is in the "sample_data" folder and that the name matches the one in the variable file (without extension). If not, amend it or reupload the file with the correct filename.


In [ ]:
file = "sample_data/india_sample"

Now let's load the file. The following bit of code will upload the file in the jupyter notebook and load the `document.xml` file inside your docx file.

In [ ]:
docxFileName = file+".docx"
docxZip = zipfile.ZipFile(docxFileName)
documentXML = docxZip.read('word/document.xml')
stylesXML = docxZip.read('word/styles.xml')
et = etree.XML(documentXML)
ns = {'w': 'http://schemas.openxmlformats.org/wordprocessingml/2006/main'}

If you want to see what are the XML files hidden in your docx file you can use: `namelist()`


In [ ]:
docxZip.namelist()

['[Content_Types].xml',
 '_rels/.rels',
 'word/_rels/document.xml.rels',
 'word/document.xml',
 'word/footer1.xml',
 'word/footnotes.xml',
 'word/endnotes.xml',
 'word/footer2.xml',
 'word/theme/theme1.xml',
 'word/settings.xml',
 'word/numbering.xml',
 'word/styles.xml',
 'word/webSettings.xml',
 'docProps/core.xml',
 'docProps/app.xml',
 'customXml/itemProps1.xml',
 'customXml/item1.xml',
 'customXml/_rels/item1.xml.rels',
 'word/fontTable.xml']

## What is XPATH?
For the purpose of this workshop we won't go into the details of xpath, but for more information you can read:
https://www.w3schools.com/xml/xml_xpath.asp

Suffice to say, in an xml, xpath tells the script where to find the information we are looking after.

In [ ]:
#Xpaths based on the paragraph/character styles defined in our word file.
#Please note that styles should have the same name as those defined at the beginning of this tutorial.

#paragrah unit
p = './w:r//w:t'

#charachter styles
ReferenceOld_xpath = './w:r[w:rPr[w:rStyle[@w:val="ReferenceOld"]]]/w:t'
ReferenceNew_xpath = './w:r[w:rPr[w:rStyle[@w:val="ReferenceNew"]]]/w:t'
PhysicalDescription_xpath = './w:r[w:rPr[w:rStyle[@w:val="PhysicalDescription"]]][w:t]/w:t'
date_xpath = './w:r[w:rPr[w:rStyle[@w:val="ItemDate"]]][w:t]/w:t'

#paragraph styles
ContentDescription_xpath = './w:pPr[w:pStyle[@w:val="ContentDescription"]]/following-sibling::w:r/w:t'
title_xpath = './w:pPr[w:pStyle[@w:val="Title"]]/following-sibling::w:r/w:t'


Now we upload the document and let the script export the information in a pandas dataframe (tabular data).

In [ ]:
table = pd.DataFrame()

#initialisation

ReferenceOld_dic = {}
ReferenceNew_dic = {}
title_dic = {}
ContentDescription_dic = {}
PhysicalDescription_dic = {}
date_dic = {}

ReferenceOld = float('NaN')
ReferenceNew = float('NaN')
title = ''
PhysicalDescription = ''
ContentDescription = ''
date = ''

#functions

def get_info(para, xpath, field):
    if para.xpath(xpath, namespaces=ns):
        text = para.xpath(xpath, namespaces=ns)
        field = " ".join([t.text.strip() for t in text]).strip('\n')
    return field
  

def populate_series(key, value, dictionary, mode):
    if key not in dictionary:
        dictionary[key] = value
    if mode=='content': 
        if value not in dictionary[key]:
            dictionary[key]+="\n"+value
            value = ''
    return dictionary, value
    

for i, para in enumerate(et.xpath('//w:p', namespaces=ns)):
      
    ###extract info from word
    
    ReferenceOld = get_info(para, ReferenceOld_xpath, ReferenceOld)
    ReferenceNew = get_info(para, ReferenceNew_xpath, ReferenceNew)
    title = get_info(para, title_xpath, title)
    PhysicalDescription = get_info(para, PhysicalDescription_xpath, PhysicalDescription)
    ContentDescription = get_info(para, ContentDescription_xpath, ContentDescription)
    date = get_info(para, date_xpath, date)
           
    ###create series
   
    ReferenceOld_dic, ReferenceOld = populate_series(ReferenceOld, ReferenceOld, ReferenceOld_dic, "ref")
    ReferenceNew_dic, ReferenceNew = populate_series(ReferenceOld, ReferenceNew, ReferenceNew_dic, "ref")
    title_dic, title = populate_series(ReferenceOld, title, title_dic, "content")
    date_dic, date = populate_series(ReferenceOld, date, date_dic, "content")
    ContentDescription_dic, ContentDescription = populate_series(ReferenceOld, ContentDescription, ContentDescription_dic, "content")
    PhysicalDescription_dic, PhysicalDescription = populate_series(ReferenceOld, PhysicalDescription, PhysicalDescription_dic, "content")

            

table = pd.DataFrame.from_dict({'ReferenceOld':pd.Series(ReferenceOld_dic),'ReferenceNew':pd.Series(ReferenceNew_dic),'Title':pd.Series(title_dic), 'PhysicalDescription':pd.Series(PhysicalDescription_dic), 'ContentDescription':pd.Series(ContentDescription_dic), 'Item date':pd.Series(date_dic)}
)

table = table.applymap(lambda x: x.strip('\n') if type(x)==str else x)

#add Language and Creator columns
header_list = ['ReferenceOld','ReferenceNew', 'Title', 'PhysicalDescription','ContentDescription', 'Item date','Language','Creator']
table = table.reindex(columns = header_list) 

#remove rows without any ReferenceOld and ReferenceNew
table = table.dropna(how='all', subset=['ReferenceOld', 'ReferenceNew'])

#save the file as a .csv
table.to_csv(file+".csv", encoding='utf-8-sig') 

#print the table 
table

,ReferenceOld,ReferenceNew,Title,PhysicalDescription,ContentDescription,Item date,Language,Creator
1,1,Mss. Eur. G. 1 .,"[“ Rec d from Exam rs Office"" 5 Oct. 1814.]\...","50 x 30 cm, pp. 264.","There is no general title, but the manuscript ...",1914 and 1919.,NaN,NaN
2,2,Mss. Eur. D. 2,A Decree of the Holy Congregation Generall for...,"30 x 18.5 cm, ' Foll . 3.","The watermarks are (a) Arms, Quarterly: 1st an...",,NaN,NaN
3,3,MSS. Eur. G. 2 .,Peticion of ye East India Company.,"44 x 32' cm. One sheet, framed and hung in the...",This document is reproduced (actual size) in R...,,NaN,NaN
4,4,MSS.Eur . D.3 .,"[Purchased 14 July 1916.]\n[ Batavia's , Statu...","33 x '21 cm. pp., vi, 200","This volume is lettered "" Batasia's Statut B...",,NaN,NaN
5,5,MSS Eur F.1,[JOSIAH WEBBE?],39 x 25 cm. pp. 126.,"Verbael , uijt afgesonden en aengekomen brie...",,NaN,NaN
